## Chargement et préparation des données.

### Initialiser Spark

In [154]:
from modules.spark import spark

In [155]:
from pyspark.sql import functions as F

- Chargement de données

In [156]:
df = spark.read.csv('../data/raw/DataCoSupplyChainDataset.csv', header=True, inferSchema=True)

### Effectuer un nettoyage initial.

In [157]:
df.printSchema()

root
 |-- Type: string (nullable = true)
 |-- Days for shipping (real): integer (nullable = true)
 |-- Days for shipment (scheduled): integer (nullable = true)
 |-- Benefit per order: double (nullable = true)
 |-- Sales per customer: double (nullable = true)
 |-- Delivery Status: string (nullable = true)
 |-- Late_delivery_risk: integer (nullable = true)
 |-- Category Id: integer (nullable = true)
 |-- Category Name: string (nullable = true)
 |-- Customer City: string (nullable = true)
 |-- Customer Country: string (nullable = true)
 |-- Customer Email: string (nullable = true)
 |-- Customer Fname: string (nullable = true)
 |-- Customer Id: integer (nullable = true)
 |-- Customer Lname: string (nullable = true)
 |-- Customer Password: string (nullable = true)
 |-- Customer Segment: string (nullable = true)
 |-- Customer State: string (nullable = true)
 |-- Customer Street: string (nullable = true)
 |-- Customer Zipcode: integer (nullable = true)
 |-- Department Id: integer (nullable = 

- Les dimensions de dataframes

In [158]:
print("Colonnes :", len(df.columns))
print("Lignes   :", df.count())

Colonnes : 53
Lignes   : 180519


- Afficher les 10 premiere lignes

In [159]:
df.show(10)

+--------+------------------------+-----------------------------+-----------------+------------------+-----------------+------------------+-----------+--------------+-------------+----------------+--------------+--------------+-----------+--------------+-----------------+----------------+--------------+--------------------+----------------+-------------+---------------+-----------+------------+------------+----------+-------------+-----------------+-----------------------+--------+----------------------+-------------------+------------------------+-------------+------------------------+-----------------------+-------------------+------+----------------+----------------------+--------------+---------------+---------------+-------------+---------------+-------------------+-------------------+--------------------+------------+-------------+--------------+--------------------------+--------------+
|    Type|Days for shipping (real)|Days for shipment (scheduled)|Benefit per order|Sales per 

- Extraction du nom de mois a partir de la colonne "order date (DateOrders)"

In [160]:
df = df.withColumn(
    "order date (DateOrders)",
    F.to_timestamp("order date (DateOrders)", "M/d/yyyy H:mm")
).withColumn("Order_Month_Name", F.date_format("order date (DateOrders)", "MMMM"))

- Extraction du nom de mois a partir de la colonne "shipping date (DateOrders)"

In [161]:
df = df.withColumn(
    "shipping date (DateOrders)",
    F.to_timestamp("shipping date (DateOrders)", "M/d/yyyy H:mm")
).withColumn("Shipping_Month_Name", F.date_format("shipping date (DateOrders)", "MMMM"))

In [162]:
pdf = df.toPandas()

pdf.isnull().sum()

Type                                  0
Days for shipping (real)              0
Days for shipment (scheduled)         0
Benefit per order                     0
Sales per customer                    0
Delivery Status                       0
Late_delivery_risk                    0
Category Id                           0
Category Name                         0
Customer City                         0
Customer Country                      0
Customer Email                        0
Customer Fname                        0
Customer Id                           0
Customer Lname                        8
Customer Password                     0
Customer Segment                      0
Customer State                        0
Customer Street                       0
Customer Zipcode                      3
Department Id                         0
Department Name                       0
Latitude                              0
Longitude                             0
Market                                0


In [163]:
pdf = pdf.drop("Days for shipping (real)", axis=1)
pdf = pdf.drop("Delivery Status", axis=1)
pdf = pdf.drop("shipping date (DateOrders)", axis=1)
pdf = pdf.drop("order date (DateOrders)", axis=1)
pdf = pdf.drop("Order Zipcode", axis=1)
pdf = pdf.drop("Customer City", axis=1)
pdf = pdf.drop("Customer Country", axis=1)
pdf = pdf.drop("Customer Email", axis=1)
pdf = pdf.drop("Customer Fname", axis=1)
pdf = pdf.drop("Customer Id", axis=1)
pdf = pdf.drop("Customer Lname", axis=1)
pdf = pdf.drop("Customer Password", axis=1)
pdf = pdf.drop("Product Image", axis=1)
pdf = pdf.drop("Product Name", axis=1)
pdf = pdf.drop("Product Description", axis=1)
pdf = pdf.drop("Product Status", axis=1)


In [164]:
len(pdf.columns)

39

In [165]:
pdf["Type"].count()

np.int64(180519)

In [166]:
pdf = pdf[~pdf.isnull().any(axis=1)]

In [167]:
pdf.isnull().sum()

Type                             0
Days for shipment (scheduled)    0
Benefit per order                0
Sales per customer               0
Late_delivery_risk               0
Category Id                      0
Category Name                    0
Customer Segment                 0
Customer State                   0
Customer Street                  0
Customer Zipcode                 0
Department Id                    0
Department Name                  0
Latitude                         0
Longitude                        0
Market                           0
Order City                       0
Order Country                    0
Order Customer Id                0
Order Id                         0
Order Item Cardprod Id           0
Order Item Discount              0
Order Item Discount Rate         0
Order Item Id                    0
Order Item Product Price         0
Order Item Profit Ratio          0
Order Item Quantity              0
Sales                            0
Order Item Total    

- Detection de l'importance des colonnes utilisant RandomForest

In [168]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np

# --- Example Data ---

X = pdf.drop('Late_delivery_risk', axis=1)
y = pdf['Late_delivery_risk']

# --- Identify numeric and categorical columns ---
numeric_cols = X.select_dtypes(include=['int32', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(exclude=['int32', 'float64']).columns.tolist()

# --- Preprocessing ---

numeric_transformer = SimpleImputer(strategy='median')

categorical_transformer = Pipeline(steps=[
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# --- Pipeline with RandomForest ---
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))
])

model.fit(X, y)


,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [169]:
# --- Feature importance ---
feature_names = X.columns
importances = model.named_steps['classifier'].feature_importances_

print(len(feature_names))
print(len(importances))

import pandas as pd
fi = pd.DataFrame({'Feature': feature_names, 'Importance': importances})


38
38


In [170]:
fi.sort_values("Importance", ascending=False)

,Feature,Importance
35,Shipping Mode,0.108172
0,Type,0.102240
34,Product Price,0.062188
27,Order Item Total,0.048204
8,Customer Street,0.047329
6,Customer Segment,0.045562
30,Order State,0.042748
9,Customer Zipcode,0.041151
13,Longitude,0.040921
7,Customer State,0.040688


- Supprimer les colonnes non pertinent

On garde:
- Type
- Shipping Mode
- Product Price
- Late_delivery_risk
- Category Name
- Customer Segment
- Order Item Quantity
- Order Region
- Shipping_Month_Name

In [ ]:
keep_cols = [
    'Type',
    'Shipping Mode',
    'Product Price',
    'Late_delivery_risk',
    'Category Name',
    'Customer Segment',
    'Order Item Quantity',
    'Order Region',
    'Shipping_Month_Name',
]

In [186]:
df_cleaned = df.select(keep_cols)

- Transformation de colonnes

In [187]:
dict_rename = {
    "Shipping Mode": "ShippingMode",
    "Product Price": "ProductPrice",
    "Late_delivery_risk": "LateDeliveryRisk",
    "Category Name": "CategoryName",
    "Customer Segment": "CustomerSegment",
    "Order Item Quantity": "OrderItemQuantity",
    "Order Region": "OrderRegion",
    "Shipping_Month_Name": "ShippingMonthName",
}

for col, new_col in dict_rename.items():
    df_cleaned = df_cleaned.withColumn(new_col, F.col(col)).drop(col)

- Sauvegarder le modéle

In [ ]:
df_cleaned.toPandas().to_csv("../data/processed/data-with-relevant-columns.csv", index=False)